In [12]:
# === NEW IMPORTS THAT ACTUALLY WORK ===
from langchain_core.documents import Document  # ← CHANGED
from langchain_text_splitters import RecursiveCharacterTextSplitter  # ← CHANGED  
from langchain_community.retrievers import BM25Retriever
from smolagents import Tool, CodeAgent, InferenceClientModel

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=InferenceClientModel(token = "hf_token"))

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options.      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Retrieve entertainment ideas                                                                                   
  entertainment_ideas = party_planning_retriever(query="luxury superhero-themed party entertainment")              
                                                                                                                   
  # Retrieve catering ideas                                                                                        
  catering_ideas = party_planning_retriever(query="luxury superhero-themed party catering")                        
                                                                                                                   
  # Retrieve decoration ideas                                                                                      
  decoration_ideas = party_planning_retriever(query="luxury superhero-themed party decorations")                   
                                                                                                                   
  # Print out the ideas for each category to organize and prepare the final answer                                 
  print("Entertainment Ideas:", entertainment_ideas)                                                               
  print("Catering Ideas:", catering_ideas)                                                                         
  print("Decoration Ideas:", decoration_ideas)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Entertainment Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.
Catering Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.
Decoration Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Out: None

[Step 1: Duration 7.79 seconds| Input tokens: 2,103 | Output tokens: 163]

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Selecting unique and appropriate ideas for each category                                                       
  selected_entertainment = "Interactive experiences with VR where guests can engage in superhero simulations or    
  compete in themed games."                                                                                        
  selected_catering = "Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's      
  Power Steak.'"                                                                                                   
  selected_decoration = "Decorate with iconic superhero logos and projections of Gotham and other superhero        
  cities around the venue, including gold accents and velvet curtains."                                            
                                                                                                                   
  # Preparing the final answer                                                                                     
  final_party_ideas = {                                                                                            
      "Entertainment": selected_entertainment,                                                                     
      "Catering": selected_catering,                                                                               
      "Decorations": selected_decoration                                                                           
  }                                                                                                                
                                                                                                                   
  # Printing the final ideas                                                                                       
  print(final_party_ideas)                                                                                         
                                                                                                                   
  # Providing the final answer                                                                                     
  final_answer(final_party_ideas)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'Entertainment': 'Interactive experiences with VR where guests can engage in superhero simulations or compete in 
themed games.', 'Catering': "Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's
Power Steak.'", 'Decorations': 'Decorate with iconic superhero logos and projections of Gotham and other superhero 
cities around the venue, including gold accents and velvet curtains.'}

Final answer: {'Entertainment': 'Interactive experiences with VR where guests can engage in superhero simulations 
or compete in themed games.', 'Catering': "Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' 
and 'Iron Man's Power Steak.'", 'Decorations': 'Decorate with iconic superhero logos and projections of Gotham and 
other superhero cities around the venue, including gold accents and velvet curtains.'}

[Step 2: Duration 9.20 seconds| Input tokens: 4,968 | Output tokens: 369]

{'Entertainment': 'Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.', 'Catering': "Serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", 'Decorations': 'Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue, including gold accents and velvet curtains.'}


In [4]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 442.0 kB/s  0:00:05 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 407.6 kB/s  0:00:04 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 436.8 kB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 350.8 kB/s  0:00:09 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 822.2 kB/s  0:00:00m-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [langchain-community]ngchain-community]
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install langchain-text-splitters


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.
